In [0]:
# Show all the files. 
display(dbutils.fs.ls("/FileStore/tables"))

path,name,size,modificationTime
dbfs:/FileStore/tables/CT_fires_2015.csv,CT_fires_2015.csv,23357531,1714442141000
dbfs:/FileStore/tables/CT_fires_2015Copy.csv,CT_fires_2015Copy.csv,23357531,1714442141000
dbfs:/FileStore/tables/employee_churn_data.csv,employee_churn_data.csv,804108,1714708580000
dbfs:/FileStore/tables/food_prices.parquet,food_prices.parquet,1394,1714442141000
dbfs:/FileStore/tables/northwind.zip,northwind.zip,52567,1714450770000
dbfs:/FileStore/tables/species.csv,species.csv,1605,1714448680000
dbfs:/FileStore/tables/surveys.csv,surveys.csv,1021588,1714448680000
dbfs:/FileStore/tables/user_device.csv,user_device.csv,17780,1714447136000
dbfs:/FileStore/tables/user_usage.csv,user_usage.csv,10088,1714447136000
dbfs:/FileStore/tables/vehicles-1.csv,vehicles-1.csv,115450,1714444956000


In [0]:
# Load the vehicles.csv file into a DataFrame.
file_location = '/FileStore/tables/employee_churn_data.csv'
df = spark.read.csv(file_location, inferSchema=True, header=True)
# Display the DataFrame.
df.show(10)

+----------+--------+------------------+--------+------+------+------------------+-----+------------------+----+
|department|promoted|            review|projects|salary|tenure|      satisfaction|bonus|     avg_hrs_month|left|
+----------+--------+------------------+--------+------+------+------------------+-----+------------------+----+
|operations|       0|0.5775686596355698|       3|   low|   5.0|0.6267589740293295|    0| 180.8660696668475|  no|
|operations|       0|0.7518996607137884|       3|medium|   6.0|0.4436789547574034|    0| 182.7081489616225|  no|
|   support|       0|0.7225484486077993|       3|medium|   6.0|0.4468232240377964|    0| 184.4160840365652|  no|
| logistics|       0|0.6751583086657402|       4|  high|   8.0|0.4401387461171622|    0| 188.7075447757313|  no|
|     sales|       0| 0.676203174778892|       3|  high|   5.0|0.5776074456916579|    1|179.82108327312108|  no|
|        IT|       0|0.6832063469509659|       2|medium|   5.0|0.5652518631559001|    1| 178.841

In [0]:
# Get the number of each salary range using a groupby. 
count_df = df.groupBy("salary").count()
count_df.show()

+------+-----+
|salary|count|
+------+-----+
|   low| 1381|
|  high| 1548|
|medium| 6611|
+------+-----+



In [0]:
# Display the DataFrame and create a bar chart.
display(count_df)

salary,count
low,1381
high,1548
medium,6611


Databricks visualization. Run in Databricks to view.

In [0]:
# Create a temporary view of the DataFrame
df.createOrReplaceTempView("employee_data")


In [0]:
%sql
--  Query that shows how many employees left the company, acoording to their salary.
SELECT salary, COUNT(*) AS num_employees_left
FROM employee_data
WHERE left = 'yes'
GROUP BY salary;

salary,num_employees_left
low,391
high,442
medium,1951


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Query to see if work shifts are a factor why employees left the company 
SELECT 
    salary,
    MIN(avg_hrs_month) AS min_hours_month,
    MAX(avg_hrs_month) AS max_hours_month,
    AVG(avg_hrs_month) AS avg_hours_month
FROM 
    employee_data
WHERE left = 'yes' 
GROUP BY salary;


salary,min_hours,max_hours,avg_hours
low,174.1457191627814,192.0486089060469,184.80249472873956
high,173.02771625356567,193.2550342661412,184.6563604171455
medium,171.37406048199938,192.9692229634878,184.7174994603533


In [0]:
%sql
-- Query to see how many employees who were promoted left the company. 
SELECT salary, COUNT(*) AS num_employees_left
FROM employee_data
WHERE left = 'yes' AND promoted = '1'
GROUP BY salary;

salary,num_employees_left
low,4
high,9
medium,44


In [0]:
%sql
-- Query to see how many employees  weren't promoted left the company. 
SELECT salary, COUNT(*) AS num_employees_left
FROM employee_data
WHERE left = 'yes' AND promoted = '0'
GROUP BY salary;

salary,num_employees_left
low,387
high,433
medium,1907


In [0]:
%sql
-- Query to see the average reviews and satisfaction of the employees who didn't left the company, gruping by "salary"
 
 SELECT 
    salary,
    AVG(review) AS avg_review,
    AVG(satisfaction) AS avg_satisfaction
FROM 
    employee_data
WHERE 
    left = 'yes' AND promoted = '1'
GROUP BY 
    salary;


salary,avg_review,avg_satisfaction
low,0.712857083566572,0.518096435607686
high,0.721680111999996,0.43421349391398767
medium,0.7075340150931049,0.46680270146456576


In [0]:
%sql
-- Query to see the average reviews and satisfaction of the employees who left the company, gruping by "salary"
 
 SELECT 
    salary,
    AVG(review) AS avg_review,
    AVG(satisfaction) AS avg_satisfaction
FROM 
    employee_data
WHERE 
    left = 'yes' AND promoted = '0'
GROUP BY 
    salary;

salary,avg_review,avg_satisfaction
low,0.6952636163993403,0.5047092200111343
high,0.6950327933509489,0.5039816541263762
medium,0.690489303460667,0.5024542207022337


In [0]:
%sql
-- Query to see how many employees who left the company were from which department and whether or not they were promoted before leaving the company

SELECT 
    department,
    SUM(CASE WHEN left = 'yes' THEN 1 ELSE 0 END) AS num_employees_left,
    SUM(CASE WHEN left = 'yes' AND promoted = 1 THEN 1 ELSE 0 END) AS num_employees_left_promoted,
    SUM(CASE WHEN left = 'yes' AND promoted = 0 THEN 1 ELSE 0 END) AS num_employees_left_no_promoted,
    COUNT(*) AS total_employees
FROM employee_data
GROUP BY department
ORDER BY num_employees_left DESC;




department,num_employees_left,num_employees_left_promoted,num_employees_left_no_promoted,total_employees
sales,537,14,523,1883
retail,471,12,459,1541
engineering,437,6,431,1516
operations,436,6,430,1522
marketing,243,5,238,802
support,212,4,208,735
admin,119,5,114,423
logistics,111,1,110,360
IT,110,1,109,356
finance,108,3,105,402
